In [ ]:
import os
import pandas as pd

DATA = "data"  # adapte si besoin
Y_TRAIN = os.path.join(DATA, "Y_train_1rknArQ.csv")

# Fichiers attendus
FILES = {
    "train_home_team": os.path.join(DATA, "train_home_team_statistics_df.csv"),
    "train_away_team": os.path.join(DATA, "train_away_team_statistics_df.csv"),
    "train_home_player": os.path.join(DATA, "train_home_player_statistics_df.csv"),
    "train_away_player": os.path.join(DATA, "train_away_player_statistics_df.csv"),
    "test_home_team": os.path.join(DATA, "test_home_team_statistics_df.csv"),
    "test_away_team": os.path.join(DATA, "test_away_team_statistics_df.csv"),
    "test_home_player": os.path.join(DATA, "test_home_player_statistics_df.csv"),
    "test_away_player": os.path.join(DATA, "test_away_player_statistics_df.csv"),
}

def read_optional(path: str) -> pd.DataFrame | None:
    return pd.read_csv(path) if os.path.exists(path) else None

def detect_key(*dfs: pd.DataFrame) -> str:
    """Trouve une clé commune (GAME_ID/MATCH_ID/...) de façon heuristique."""
    commons = None
    for df in dfs:
        if df is None: 
            continue
        cols = set(df.columns)
        commons = cols if commons is None else commons & cols
    if not commons:
        raise ValueError("Aucune colonne commune trouvée pour la jointure.")
    # score les colonnes probables
    def score(c: str) -> int:
        cl = c.lower()
        s = 0
        if "game" in cl or "match" in cl: s += 3
        if "id" in cl: s += 2
        return s
    candidates = sorted(list(commons), key=lambda c: (score(c), c.lower()), reverse=True)
    return candidates[0]

def add_prefix_except_key(df: pd.DataFrame, prefix: str, key: str) -> pd.DataFrame:
    if df is None:
        return None
    ren = {c: f"{prefix}{c}" for c in df.columns if c != key}
    return df.rename(columns=ren)

def keep_numeric_plus_id(df: pd.DataFrame, id_col: str) -> pd.DataFrame:
    num = df.select_dtypes(include=["number"]).astype("float32")
    if id_col in df.columns and id_col not in num.columns:
        num = pd.concat([df[[id_col]], num], axis=1)
    return num

# --- Chargement
th = pd.read_csv(FILES["train_home_team"])
ta = pd.read_csv(FILES["train_away_team"])
tph = read_optional(FILES["train_home_player"])
tpa = read_optional(FILES["train_away_player"])

vh = pd.read_csv(FILES["test_home_team"])
va = pd.read_csv(FILES["test_away_team"])
vph = read_optional(FILES["test_home_player"])
vpa = read_optional(FILES["test_away_player"])

# --- Détection clé
key = detect_key(th, ta, tph, tpa, vh, va, vph, vpa)
print(f"🔑 Clé détectée: {key}")

# --- Préfixes
th, ta = add_prefix_except_key(th, "home_team_", key), add_prefix_except_key(ta, "away_team_", key)
if tph is not None: tph = add_prefix_except_key(tph, "home_player_", key)
if tpa is not None: tpa = add_prefix_except_key(tpa, "away_player_", key)
vh, va = add_prefix_except_key(vh, "home_team_", key), add_prefix_except_key(va, "away_team_", key)
if vph is not None: vph = add_prefix_except_key(vph, "home_player_", key)
if vpa is not None: vpa = add_prefix_except_key(vpa, "away_player_", key)

# --- Merge TRAIN
train = th.merge(ta, on=key, how="inner", validate="one_to_one")
if tph is not None:
    train = train.merge(tph, on=key, how="left")
if tpa is not None:
    train = train.merge(tpa, on=key, how="left")

# --- Merge TEST
test = vh.merge(va, on=key, how="inner", validate="one_to_one")
if vph is not None:
    test = test.merge(vph, on=key, how="left")
if vpa is not None:
    test = test.merge(vpa, on=key, how="left")

# Uniformise l'ID
if key != "id":
    train = train.rename(columns={key: "id"})
    test = test.rename(columns={key: "id"})
id_col = "id"

# Nettoyage colonnes vides/constantes (optionnel light)
to_drop = [c for c in train.columns if c != id_col and train[c].nunique(dropna=False) <= 1]
if to_drop:
    train = train.drop(columns=to_drop)
    test = test.drop(columns=[c for c in to_drop if c in test.columns], errors="ignore")

# Ne garder que numérique + id
train = keep_numeric_plus_id(train, id_col)
test  = keep_numeric_plus_id(test, id_col)

print("✅ shapes:", train.shape, test.shape)

# Sauvegarde X
xtrain_path = os.path.join(DATA, "x_train_merged.csv")
xtest_path  = os.path.join(DATA, "x_test_merged.csv")
train.to_csv(xtrain_path, index=False)
test.to_csv(xtest_path, index=False)
print(f"💾 Écrit: {xtrain_path} & {xtest_path}")

# Sauvegarde y (aligné)
if os.path.exists(Y_TRAIN):
    y = pd.read_csv(Y_TRAIN)
    # détecte id + colonnes cible
    id_guess = None
    for cand in ["id", "row_id", "match_id", "game_id", "MATCH_ID", "GAME_ID"]:
        if cand in y.columns:
            id_guess = cand; break
    if id_guess is None:
        id_guess = y.columns[0]
    # nommage propre
    cols = {id_guess: "id"}
    for c in y.columns:
        cl = c.lower()
        if "home" in cl and "prob" not in cl: cols[c] = "home"
        if "draw" in cl: cols[c] = "draw"
        if "away" in cl: cols[c] = "away"
    y = y.rename(columns=cols)
    y = y[["id","home","draw","away"]].copy()
    # filtre aux ids présents dans X train
    y = y[y["id"].isin(train["id"])]
    y.to_csv(os.path.join(DATA, "y_train_aligned.csv"), index=False)
    print("💾 Écrit: data/y_train_aligned.csv")
else:
    print("⚠️ data/Y_train_1rknArQ.csv introuvable → pas de y aligné.")


In [ ]:
y_train = pd.read_csv("../data/Y_train_1rknArQ.csv")
print(y_train.head())


In [ ]:
#connaitre la clé de jointure 
import os
import pandas as pd
DATA = "../data"
Y_TRAIN = os.path.join(DATA, "Y_train_1rknArQ.csv")

FILES = {
    "train_home_team": os.path.join(DATA, "train_home_team_statistics_df.csv"),
    "train_away_team": os.path.join(DATA, "train_away_team_statistics_df.csv"),
    "train_home_player": os.path.join(DATA, "train_home_player_statistics_df.csv"),
    "train_away_player": os.path.join(DATA, "train_away_player_statistics_df.csv"),
}
# Lis les 4 tables du train
home_team = pd.read_csv(FILES["train_home_team"])
away_team = pd.read_csv(FILES["train_away_team"])
home_player = pd.read_csv(FILES["train_home_player"])
away_player = pd.read_csv(FILES["train_away_player"])


# Affiche les noms de colonnes
print("🏠 Home team columns:", list(home_team.columns)[:10])
print("🚩 Away team columns:", list(away_team.columns)[:10])
print("👤 Home player columns:", list(home_player.columns)[:10])
print("👥 Away player columns:", list(away_player.columns)[:10])

# Trouve les colonnes communes
common = set(home_team.columns) & set(away_team.columns) & set(home_player.columns) & set(away_player.columns)
print("\n🔑 Colonnes communes entre les 4 fichiers:", common)


🏠 Home team columns: ['ID', 'LEAGUE', 'TEAM_NAME', 'TEAM_SHOTS_TOTAL_season_sum', 'TEAM_SHOTS_INSIDEBOX_season_sum', 'TEAM_SHOTS_OFF_TARGET_season_sum', 'TEAM_SHOTS_ON_TARGET_season_sum', 'TEAM_SHOTS_OUTSIDEBOX_season_sum', 'TEAM_PASSES_season_sum', 'TEAM_SUCCESSFUL_PASSES_season_sum']
🚩 Away team columns: ['ID', 'LEAGUE', 'TEAM_NAME', 'TEAM_SHOTS_TOTAL_season_sum', 'TEAM_SHOTS_INSIDEBOX_season_sum', 'TEAM_SHOTS_OFF_TARGET_season_sum', 'TEAM_SHOTS_ON_TARGET_season_sum', 'TEAM_SHOTS_OUTSIDEBOX_season_sum', 'TEAM_PASSES_season_sum', 'TEAM_SUCCESSFUL_PASSES_season_sum']
👤 Home player columns: ['ID', 'LEAGUE', 'TEAM_NAME', 'POSITION', 'PLAYER_NAME', 'PLAYER_ACCURATE_CROSSES_season_sum', 'PLAYER_ACCURATE_PASSES_season_sum', 'PLAYER_AERIALS_WON_season_sum', 'PLAYER_ASSISTS_season_sum', 'PLAYER_BIG_CHANCES_CREATED_season_sum']
👥 Away player columns: ['ID', 'LEAGUE', 'TEAM_NAME', 'POSITION', 'PLAYER_NAME', 'PLAYER_ACCURATE_CROSSES_season_sum', 'PLAYER_ACCURATE_PASSES_season_sum', 'PLAYER_AERIA

In [6]:
def check_key(df, name):
    n = len(df)
    u = df['ID'].nunique()
    print(f"{name}: {u}/{n} IDs uniques")
    if u != n:
        print(" -> Plusieurs lignes par match (ID)")

check_key(home_team,  "home_team")
check_key(away_team,  "away_team")
check_key(home_player,"home_player")
check_key(away_player,"away_player")


home_team: 12303/12303 IDs uniques
away_team: 12303/12303 IDs uniques
home_player: 12303/237079 IDs uniques
 -> Plusieurs lignes par match (ID)
away_player: 12303/236132 IDs uniques
 -> Plusieurs lignes par match (ID)


In [7]:
import pandas as pd

def aggregate_player_stats(df, prefix):
    """Agrège les stats d'un fichier joueur au niveau du match (ID)."""
    # garder seulement les colonnes numériques
    num = df.select_dtypes(include=['number']).copy()
    if 'ID' not in num.columns:
        num = num.join(df['ID'])
    num = num.set_index('ID')

    # Calcul des statistiques agrégées (moyenne, somme et écart-type)
    agg = num.groupby(level=0).agg(['mean', 'sum', 'std'])
    # aplatir les multi-index
    agg.columns = [f"{prefix}{c}_{stat}" for c, stat in agg.columns]
    agg = agg.reset_index()
    return agg

home_player_agg = aggregate_player_stats(home_player, "home_player_")
away_player_agg = aggregate_player_stats(away_player, "away_player_")


In [9]:
# Renommer les colonnes team pour éviter les collisions
def rename_with_prefix(df, prefix):
    return df.rename(columns={c: f"{prefix}{c}" for c in df.columns if c != "ID"})

home_team_ren = rename_with_prefix(home_team, "home_team_")
away_team_ren = rename_with_prefix(away_team, "away_team_")

# Jointure sur ID
train_df = (
    home_team_ren
    .merge(away_team_ren, on="ID", how="inner")
    .merge(home_player_agg, on="ID", how="left")
    .merge(away_player_agg, on="ID", how="left")
)

print("✅ Fusion complète :", train_df.shape)
train_df.to_csv("../data/x_train_full.csv", index=False)


✅ Fusion complète : (12303, 2097)


In [10]:
train_df.head()
train_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12303 entries, 0 to 12302
Columns: 2097 entries, ID to away_player_PLAYER_SHOTS_OFF_TARGET_5_last_match_std_std
dtypes: float64(2092), int64(1), object(4)
memory usage: 196.8+ MB


In [11]:
text_cols = train_df.select_dtypes(include=['object']).columns
print("Colonnes texte :", text_cols.tolist())


Colonnes texte : ['home_team_LEAGUE', 'home_team_TEAM_NAME', 'away_team_LEAGUE', 'away_team_TEAM_NAME']


In [12]:
train_df = train_df.drop(columns=text_cols)


In [14]:
train_df.to_csv("../data/x_train_clean.csv", index=False)
print("✅ x_train_clean.csv prêt pour l'entraînement :", train_df.shape)


✅ x_train_clean.csv prêt pour l'entraînement : (12303, 2093)


In [16]:
home_team_test = pd.read_csv("../data/test_home_team_statistics_df.csv")
away_team_test = pd.read_csv("../data/test_away_team_statistics_df.csv")
home_player_test = pd.read_csv("../data/test_home_player_statistics_df.csv")
away_player_test = pd.read_csv("../data/test_away_player_statistics_df.csv")

home_player_test_agg = aggregate_player_stats(home_player_test, "home_player_")
away_player_test_agg = aggregate_player_stats(away_player_test, "away_player_")

home_team_test_ren = rename_with_prefix(home_team_test, "home_team_")
away_team_test_ren = rename_with_prefix(away_team_test, "away_team_")

test_df = (
    home_team_test_ren
    .merge(away_team_test_ren, on="ID", how="inner")
    .merge(home_player_test_agg, on="ID", how="left")
    .merge(away_player_test_agg, on="ID", how="left")
)

print("✅ Fusion test complète :", test_df.shape)
test_df.to_csv("../data/x_test_full.csv", index=False)



✅ Fusion test complète : (25368, 2093)


In [17]:
print("Train matches:", train_df["ID"].nunique())
print("Test matches:", test_df["ID"].nunique())


Train matches: 12303
Test matches: 25368


In [19]:
import pandas as pd
import numpy as np

# Charge tes fusions complètes
train_df = pd.read_csv("../data/x_train_full.csv")
test_df  = pd.read_csv("../data/x_test_full.csv")

# 1) Drop colonnes non numériques (texte) + garder id
obj_cols = train_df.select_dtypes(include=['object']).columns.tolist()
obj_cols = [c for c in obj_cols if c != 'ID']   # on garde ID
train_df = train_df.drop(columns=obj_cols, errors='ignore')
test_df  = test_df.drop(columns=obj_cols, errors='ignore')

# 2) Renommer ID -> id
train_df = train_df.rename(columns={'ID':'id'})
test_df  = test_df.rename(columns={'ID':'id'})

# 3) Aligner EXACTEMENT les colonnes (sauf 'id')
train_cols = set(train_df.columns) - {'id'}
test_cols  = set(test_df.columns) - {'id'}
common     = ['id'] + sorted(list(train_cols & test_cols))

X_train = train_df[common].copy()
X_test  = test_df[common].copy()

print("Shapes alignées:", X_train.shape, X_test.shape)

# 4) Sauvegarde clean
X_train.to_csv("../data/x_train_clean.csv", index=False)
X_test.to_csv("../data/x_test_clean.csv", index=False)
print("✅ Sauvé: data/x_train_clean.csv, data/x_test_clean.csv")


Shapes alignées: (12303, 2093) (25368, 2093)
✅ Sauvé: data/x_train_clean.csv, data/x_test_clean.csv


In [21]:
y = pd.read_csv("../data/Y_train_1rknArQ.csv")

# détecter la colonne id et renommer
id_col = None
for cand in ["id","ID","row_id","match_id","game_id","MATCH_ID","GAME_ID"]:
    if cand in y.columns:
        id_col = cand; break
if id_col is None: id_col = y.columns[0]
y = y.rename(columns={id_col:'id'})

# s’assurer d’avoir bien les 3 colonnes probas
ren = {}
for c in y.columns:
    cl = c.lower()
    if "home" in cl and "prob" not in cl: ren[c] = "home"
    elif "draw" in cl: ren[c] = "draw"
    elif "away" in cl: ren[c] = "away"
y = y.rename(columns=ren)[["id","home","draw","away"]]

# filtrer sur les ids présents dans X_train
y = y[y["id"].isin(X_train["id"])].copy()

# classe 0/1/2 = argmax([home,draw,away])
y_target = np.argmax(y[["home","draw","away"]].values, axis=1)

# Sauvegarde
y.to_csv("../data/y_train_aligned.csv", index=False)
pd.DataFrame({"id": y["id"], "target": y_target}).to_csv("../data/y_target.csv", index=False)
print("✅ Sauvé: ../data/y_train_aligned.csv, ../data/y_target.csv", y.shape, y_target.shape)


✅ Sauvé: ../data/y_train_aligned.csv, ../data/y_target.csv (12303, 4) (12303,)
